In [ ]:
from repenseai.genai.agent import Agent
from repenseai.genai.tasks.api import Task
from repenseai.genai.tasks.parallel import ParallelTask

from pydantic import BaseModel, Field
from typing import Literal, List

# Define Pydantic models for structured outputs
class TaskDefinition(BaseModel):
    original_task: str
    task_type: Literal["formal", "conversational", "hybrid"]
    task_description: str

class TaskList(BaseModel):
    analysis: str
    tasks: List[TaskDefinition] = Field(..., default_factory=list)

# Define the prompts
ORCHESTRATOR_PROMPT = """
Analyze this task and break it down into 2-3 distinct approaches:

Task: {task}

Provide an Analysis:

Explain your understanding of the task and which variations would be valuable.
Focus on how each approach serves different aspects of the task.

Along with the analysis, provide 2-3 approaches to tackle the task, each with a brief description:

Formal style: Write technically and precisely, focusing on detailed specifications
Conversational style: Write in a friendly and engaging way that connects with the reader
Hybrid style: Tell a story that includes technical details, combining emotional elements with specifications

Return only JSON output.
"""

WORKER_PROMPT = """
Generate content based on:
Task: {original_task}
Style: {task_type}
Guidelines: {task_description}

Return only your response:
[Your content here, maintaining the specified style and fully addressing requirements.]
"""

# Define the main task
task = """Write a product description for a new eco-friendly water bottle. 
The target audience is environmentally conscious millennials and key product features are: 
    1. plastic-free
    2. insulated 
    3. lifetime warranty
"""

# Set up the orchestrator agent
orchestrator_agent = Agent(
    model="gpt-4o",
    model_type="chat",
    json_schema=TaskList,
)

# Create the orchestrator task
orchestrator_task = Task(
    user=ORCHESTRATOR_PROMPT,
    agent=orchestrator_agent,
    simple_response=True,
)

initial_results = orchestrator_task.run({'task': task})


# Print orchestrator analysis and tasks
print("\n=== ORCHESTRATOR OUTPUT ===")
print(f"\nANALYSIS:\n{initial_results['analysis']}")
print(f"\nTASKS:")
for task_info in initial_results['tasks']:
    print(f"Type: {task_info['task_type']}")
    print(f"Description: {task_info['task_description']}\n")


# Create worker tasks based on orchestrator output
worker_agent = Agent(
    model="llama-3.3-70b-versatile",
    model_type="chat",
)

output_tasks = initial_results['tasks']

worker_task = Task(
    user=WORKER_PROMPT, 
    agent=worker_agent, 
    simple_response=True
) 

# Create a parallel task for all worker tasks
parallel_worker_task = ParallelTask(tasks=worker_task)
worker_results = parallel_worker_task.run(output_tasks)

# Print worker results
for task_info, response in zip(initial_results['tasks'], worker_results):
    print(f"\n=== WORKER RESULT ({task_info['task_type']}) ===\n{response}\n")